<a href="https://colab.research.google.com/github/NoahBjongHoKim/IML_Projects_BoK/blob/Sara/template_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 3
This serves as a template which will guide you through the implementation of this task. It is advised to first read the whole template and get a sense of the overall structure of the code before trying to fill in any of the TODO gaps.
This is the jupyter notebook version of the template. For the python file version, please refer to the file `template_solution.py`.

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


First, we import necessary libraries:

just added whatever was needed, probably also stuff not needed anymore

In [2]:
import numpy as np
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
import os
import torch
import torchvision
from torchvision import transforms
import torchvision.datasets as datasets
import torch.nn as nn
import torch.nn.functional as F
from torchvision.io import read_image
from torchvision.models import resnet50, ResNet50_Weights
import torch.optim as optim
# Add any other imports you need here

In [3]:
# The device is automatically set to GPU if available, otherwise CPU
# If you want to force the device to CPU, you can change the line to
# device = torch.device("cpu")
# When using the GPU, it is important that your model and all data are on the
# same device.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

This class to get an embedding from the pretrained network and not a prediction

In [ ]:
class NoFinalLayer(nn.Module):
    def __init__(self, original_model):
        super(NoFinalLayer, self).__init__()
        self.features = nn.Sequential(*list(original_model.children())[:-1])

    def forward(self, x):
        x = self.features(x)
        return x

Reading all the stupid images and applying the pretrained network on them. Takes ages. Try to avoid.

In [ ]:
def generate_embeddings():
    """
    Transform, resize and normalize the images and then use a pretrained model to extract
    the embeddings.
    """
    # TODO: define a transform to pre-process the images
    # The required pre-processing depends on the pre-trained model you choose
    # below.
    # See https://pytorch.org/vision/stable/models.html#using-the-pre-trained-models

    #Average image width: 453.9186
    #Average image height: 306.4356
    image_size = (306,454)
    mean_per_channel = (161.6640330815327, 140.88972730355985, 111.99007629992748)
    standard_deviation_per_channel_reciprocal = (1/57.38794298481981, 1/62.1605772577997, 1/70.41062360670296)

    train_transforms = transforms.Compose([
                               transforms.Resize(image_size),
                               transforms.CenterCrop(image_size),
                               transforms.ToTensor(),
                               transforms.Normalize(mean = mean_per_channel, std= standard_deviation_per_channel_reciprocal),
                             ])

    path = "/content/drive/MyDrive/task3_full"

    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/task3_full/dataset", transform=train_transforms)
    # Hint: adjust batch_size and num_workers to your PC configuration, so that you don't
    # run out of memory (VRAM if on GPU, RAM if on CPU)

    #we have to pass images of the same size to the DataLoader or it is unhappy (just like me)
    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=64, #teiler von 10001 #maybe set to 32 if it does not work
                              shuffle=False, #set this to true?
                              pin_memory=True, num_workers=16, #and this to 8, google says 2 originally 16
                              )

    # TODO: define a model for extraction of the embeddings (Hint: load a pretrained model,
    #  more info here: https://pytorch.org/vision/stable/models.html)
    weights = ResNet50_Weights.DEFAULT
    model_init = resnet50(weights=weights)
    model = NoFinalLayer(model_init)
    model.to(device)
    embedding_size = 2048 #this works, I tried it

    # TODO: pick your model
    num_images = len(train_dataset)
    print(num_images)
    print(embedding_size)
    embeddings_list = []
    # TODO: Use the model to extract the embeddings. Hint: remove the last layers of the
    # model to access the embeddings the model generates.

    model.eval()  # Set the model to evaluation mode
    number=0
    with torch.no_grad():  # Disable gradient computation
        for images, _ in train_loader:
            #print("size :", images.size(), " type:", images.type())
            images = images.to(device)  # Move images to the device
            # Forward pass through the model to get the output
            number = number + 1
            outputs = model(images) # Get both output and auxiliary output
            # Extract embeddings from the output
            embeddings_list.append(outputs.cpu().numpy().squeeze())   # Append embeddings to the list
            print("Batch: ",number, "/",num_images/91)
    embeddings_array = np.concatenate(embeddings_list, axis=0)
    #embeddings_list = np.array(embeddings_list)
    print(embeddings_array.shape)
    assert(np.shape(embeddings_array) == (num_images, embedding_size))
    np.save("/content/drive/MyDrive/task3_full/dataset/embeddings.npy", embeddings_array)

In [ ]:
print(device)
generate_embeddings()


cuda:0


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 8, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 60.0MB/s]


10023
2048


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Batch:  1 / 110.14285714285714
Batch:  2 / 110.14285714285714
Batch:  3 / 110.14285714285714
Batch:  4 / 110.14285714285714
Batch:  5 / 110.14285714285714
Batch:  6 / 110.14285714285714
Batch:  7 / 110.14285714285714
Batch:  8 / 110.14285714285714
Batch:  9 / 110.14285714285714
Batch:  10 / 110.14285714285714
Batch:  11 / 110.14285714285714
Batch:  12 / 110.14285714285714
Batch:  13 / 110.14285714285714
Batch:  14 / 110.14285714285714
Batch:  15 / 110.14285714285714
Batch:  16 / 110.14285714285714
Batch:  17 / 110.14285714285714
Batch:  18 / 110.14285714285714
Batch:  19 / 110.14285714285714
Batch:  20 / 110.14285714285714
Batch:  21 / 110.14285714285714
Batch:  22 / 110.14285714285714
Batch:  23 / 110.14285714285714
Batch:  24 / 110.14285714285714
Batch:  25 / 110.14285714285714
Batch:  26 / 110.14285714285714
Batch:  27 / 110.14285714285714
Batch:  28 / 110.14285714285714
Batch:  29 / 110.14285714285714
Batch:  30 / 110.14285714285714
Batch:  31 / 110.14285714285714
Batch:  32 / 110.

In [8]:
def get_data(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/task3_full/dataset",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/task3_full/dataset/embeddings.npy')
    # TODO: Normalize the embeddings
    #embeddings = torch.nn.functional.normalize(embeddings)

    #Mean per channel: [161.6640330815327, 140.88972730355985, 111.99007629992748]
    #Standard deviation per channel: [1/57.38794298481981, 1/62.1605772577997, 1/70.41062360670296]
    #embeddings = torchvision.transforms.Normalize(embeddings, mean = [161.6640330815327, 140.88972730355985, 111.99007629992748],
    #                                                         std = [57.38794298481981, 62.1605772577997, 70.41062360670296])
    # Use numpy's L2 normalization
    norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
    # Normalize embeddings
    embeddings_normalized = embeddings / norm
    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings_normalized[i]
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        #if train:
            #X.append(np.hstack([emb[0], emb[2], emb[1]]))
            #y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

In [5]:
def get_data_triplet(file, train=True):
    """
    Load the triplets from the file and generate the features and labels.

    input: file: string, the path to the file containing the triplets
          train: boolean, whether the data is for training or testing

    output: X: numpy array, the features
            y: numpy array, the labels
    """
    triplets = []
    with open(file) as f:
        for line in f:
            triplets.append(line)

    # generate training data from triplets
    train_dataset = datasets.ImageFolder(root="/content/drive/MyDrive/task3_full/dataset",
                                         transform=None)
    filenames = [s[0].split('/')[-1].replace('.jpg', '') for s in train_dataset.samples]
    embeddings = np.load('/content/drive/MyDrive/task3_full/dataset/embeddings.npy')
    # TODO: Normalize the embeddings
    #embeddings = torch.nn.functional.normalize(embeddings)

    #Mean per channel: [161.6640330815327, 140.88972730355985, 111.99007629992748]
    #Standard deviation per channel: [1/57.38794298481981, 1/62.1605772577997, 1/70.41062360670296]
    #embeddings = torchvision.transforms.Normalize(embeddings, mean = [161.6640330815327, 140.88972730355985, 111.99007629992748],
    #                                                         std = [57.38794298481981, 62.1605772577997, 70.41062360670296])
    # Use numpy's L2 normalization
    norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
    # Normalize embeddings
    embeddings_normalized = embeddings / norm
    file_to_embedding = {}
    for i in range(len(filenames)):
        file_to_embedding[filenames[i]] = embeddings_normalized[i]
    X = []
    y = []
    # use the individual embeddings to generate the features and labels for triplets
    for t in triplets:
        emb = [file_to_embedding[a] for a in t.split()]
        X.append(np.hstack([emb[0], emb[1], emb[2]]))
        y.append(1)
        # Generating negative samples (data augmentation)
        #if train:
            #X.append(np.hstack([emb[0], emb[2], emb[1]]))
            #y.append(0)
    X = np.vstack(X)
    y = np.hstack(y)
    return X, y

We are given A, B, C
A is the Anchor
B is positive
C is negative
Then y = 1

If we swap B, C (so A, C, B if original)
A is the Anchor
C is negative
B is positive
Then y = 0

So given a triplet P1, P2, P3, label y+1 denotes the index of the negative (in embeddings)


Hint: adjust batch_size and num_workers to your PC configuration, so that you don't run out of memory (VRAM if on GPU, RAM if on CPU)

In [6]:
def create_loader_from_np(X, y = None, train = True, batch_size=64, shuffle=True, num_workers = 4):
    """
    Create a torch.utils.data.DataLoader object from numpy arrays containing the data.

    input: X: numpy array, the features
           y: numpy array, the labels

    output: loader: torch.data.util.DataLoader, the object containing the data
    """
    if train:
        # Attention: If you get type errors you can modify the type of the
        # labels here
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float),
                                torch.from_numpy(y).type(torch.long))
    else:
        dataset = TensorDataset(torch.from_numpy(X).type(torch.float))
    loader = DataLoader(dataset=dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        pin_memory=True, num_workers=num_workers)
    return loader

TODO: define a model. Here, the basic structure is defined, but you need to fill in the details

In [7]:
import torch.nn.init as init
class Net(nn.Module):
    """
    The model class, which defines our classifier.
    """
    def __init__(self):
        """
        The constructor of the model.
        """
        super().__init__()
        self.fc1 = nn.Linear(3*2048, 2048)
        self.fc12 = nn.Linear(2048, 2048)
        self.fc13 = nn.Linear(2048, 2048)
        self.fc14 = nn.Linear(2048, 2048)
        self.fc15 = nn.Linear(2048, 128)
        self.fc3 = nn.Linear(128, 1)

        # Print initial weights
        print("Initial weights of the linear layer:")
        print(self.fc1.weight)
        print("Initial bias of the linear layer:")
        print(self.fc1.bias)

        # Initialize weights using Xavier initialization
        init.xavier_uniform_(self.fc1.weight)
        init.constant_(self.fc1.bias, 0)  # Initialize bias to zeros

        # Print weights after initialization
        print("Weights after Xavier initialization:")
        print(self.fc1.weight)
        print("Bias after initialization:")
        print(self.fc1.bias)

    def forward(self, x):
        """
        The forward pass of the model.

        input: x: torch.Tensor, the input to the model

        output: x: torch.Tensor, the output of the model
        """
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc12(x))
        x = F.relu(self.fc13(x))
        x = F.relu(self.fc14(x))
        x = F.relu(self.fc15(x))
        x = self.fc3(x)
        return x

"""
class TripletNet(nn.Module):
    def __init__(self):
        super(TripletNet, self).__init__()
        # Define your base network here
        self.base_network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, embedding_size),
        )

    def forward(self, anchor, positive, negative):
        anchor_embedding = self.base_network(anchor)
        positive_embedding = self.base_network(positive)
        negative_embedding = self.base_network(negative)
        return anchor_embedding, positive_embedding, negative_embedding
"""

'\nclass TripletNet(nn.Module):\n    def __init__(self):\n        super(TripletNet, self).__init__()\n        # Define your base network here\n        self.base_network = nn.Sequential(\n            nn.Linear(input_size, hidden_size),\n            nn.ReLU(),\n            nn.Linear(hidden_size, embedding_size),\n        )\n\n    def forward(self, anchor, positive, negative):\n        anchor_embedding = self.base_network(anchor)\n        positive_embedding = self.base_network(positive)\n        negative_embedding = self.base_network(negative)\n        return anchor_embedding, positive_embedding, negative_embedding\n'

In [ ]:
class TripletNet(nn.Module):
    def __init__(self):
        super(TripletNet, self).__init__()
        # Define your base network here
        self.base_network = nn.Sequential(
            nn.Linear(2048, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
        )

    def forward(self, anchor, positive, negative):
        anchor_embedding = self.base_network(anchor)
        positive_embedding = self.base_network(positive)
        negative_embedding = self.base_network(negative)
        return anchor_embedding, positive_embedding, negative_embedding

In [9]:
TRAIN_TRIPLETS = '/content/drive/MyDrive/task3_full/train_triplets.txt'

# load the training data
X, y = get_data(TRAIN_TRIPLETS)
# Create data loaders for the training data
#print(X)
#print(y)
train_loader = create_loader_from_np(X, y, train = True, batch_size=32)
# delete the loaded training data to save memory, as the data loader copies
del X
del y

In [10]:
TEST_TRIPLETS = '/content/drive/MyDrive/task3_full/test_triplets.txt'

# repeat for testing data
X_test, y_test = get_data(TEST_TRIPLETS, train=False)
test_loader = create_loader_from_np(X_test, train = False, batch_size=2048, shuffle=False)
del X_test
del y_test

In [ ]:
def validation_acc(validation_pred, validation_label):
  acc = 0
  assert(validation_pred.size() == validation_label.size())
  for i in range(validation_pred.size()):
    if validation_pred[i] == validation_label[i]:
      acc+=1
  return acc/validation_pred.size()

In [ ]:
def train_model_triplet(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data

    output: model: torch.nn.Module, the trained model
    """
    model = TripletNet()
    model.train()
    model.to(device)
    n_epochs = 10
    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part
    # of the training data as a validation split. After each epoch, compute the loss on the
    # validation split and print it out. This enables you to see how your model is performing
    # on the validation data before submitting the results on the server. After choosing the
    # best model, train it on the whole training data.

    #use triplet loss?
    criterion = nn.TripletMarginLoss(margin = 0.1)  # Define your loss function here
    #criterion = nn.TripletMarginLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.1)  # Define your optimizer
    acc=0
    count=0
    #best_model =  Net()


    for epoch in range(n_epochs):
        running_loss = 0.0
        #define validation subset
        #validation =


        for i, data in enumerate(train_loader, 0):
            #print("i: ", i, " Data: ", data, "\n")
            inputs, labels = data[0].to(device), data[1].to(device)
            #print("Inputs:", inputs.size(), "Labels:", labels.size(), "\n")
            labels = labels.type(torch.float)
            anchor = inputs[:, :2048]
            #print(anchor.shape)
            positive = inputs[:,2048:4096]
            negative = inputs[:,4096:]

            anchor_embed, positive_embed, negative_embed = model(anchor, positive, negative)
            loss = criterion(anchor_embed, positive_embed, negative_embed)

            #print("anchor:", anchor, "positive:", positive, "negative:", negative, "\n")

            # Calculate the Euclidean norm between anchor and positive embeddings
            dist_positive = torch.norm(anchor_embed - positive_embed, dim=1)

            # Calculate the Euclidean norm between anchor and negative embeddings
            dist_negative = torch.norm(anchor_embed - negative_embed, dim=1)

            # Print or use the distances as needed
            #print("Euclidean distance between anchor and positive:", dist_positive)
            #print("Euclidean distance between anchor and negative:", dist_negative)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()


            # Compute accuracy
            acc = ((dist_positive >= dist_negative)==labels).sum().item()
            count = len(labels)

            running_loss += loss.item()
            if i % 200 == 199:  # Print every 20 mini-batches
                #for name, param in model.named_parameters():
                #  print(name, param)
                #print(predicted.sum())
                accper = acc/count
                print('[%d, %5d] loss: %.3f acc: %.3f' %
                      (epoch + 1, i + 1, running_loss / 200, accper))
                print((dist_positive >= dist_negative))
                running_loss = 0.0
            # Get predicted classes as the entry with the highest probability

          #score_validation = validation_acc(validation, model(validation))
          #print("Epoch :" epoch, "Score: ", score_validation, "\n")


    print('Finished Training')
    return model

In [ ]:
model=train_model_triplet(train_loader)

[1,   200] loss: 0.101 acc: 0.000
tensor([True], device='cuda:0')
[1,   400] loss: 0.103 acc: 0.000
tensor([True], device='cuda:0')
[1,   600] loss: 0.099 acc: 1.000
tensor([False], device='cuda:0')
[1,   800] loss: 0.100 acc: 0.000
tensor([True], device='cuda:0')
[1,  1000] loss: 0.097 acc: 0.000
tensor([True], device='cuda:0')
[1,  1200] loss: 0.102 acc: 0.000
tensor([False], device='cuda:0')
[1,  1400] loss: 0.097 acc: 1.000
tensor([True], device='cuda:0')
[1,  1600] loss: 0.095 acc: 0.000
tensor([True], device='cuda:0')
[1,  1800] loss: 0.102 acc: 0.000
tensor([True], device='cuda:0')
[1,  2000] loss: 0.099 acc: 0.000
tensor([True], device='cuda:0')
[1,  2200] loss: 0.095 acc: 0.000
tensor([False], device='cuda:0')
[1,  2400] loss: 0.104 acc: 1.000
tensor([False], device='cuda:0')
[1,  2600] loss: 0.099 acc: 1.000
tensor([True], device='cuda:0')
[1,  2800] loss: 0.102 acc: 1.000
tensor([True], device='cuda:0')
[1,  3000] loss: 0.102 acc: 1.000
tensor([False], device='cuda:0')
[1,  

Exception in thread Thread-148 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 53, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 30, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 495, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", line 57, in detach
    with _resource_sharer.get_connection(self._id) as conn:
  File "/usr/lib/python

KeyboardInterrupt: 

In [ ]:
def train_model(train_loader):
    """
    The training procedure of the model; it accepts the training data, defines the model
    and then trains it.

    input: train_loader: torch.data.util.DataLoader, the object containing the training data

    output: model: torch.nn.Module, the trained model
    """
    model = Net()
    model.train()
    model.to(device)
    n_epochs = 10
    # TODO: define a loss function, optimizer and proceed with training. Hint: use the part
    # of the training data as a validation split. After each epoch, compute the loss on the
    # validation split and print it out. This enables you to see how your model is performing
    # on the validation data before submitting the results on the server. After choosing the
    # best model, train it on the whole training data.

    #use triplet loss?
    criterion = nn.L1Loss()  # Define your loss function here
    #criterion = nn.TripletMarginLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)  # Define your optimizer
    acc=0
    count=0
    #best_model =  Net()


    for epoch in range(n_epochs):
        running_loss = 0.0
        #define validation subset
        #validation =


        for i, data in enumerate(train_loader, 0):
            #print("i: ", i, " Data: ", data, "\n")
            inputs, labels = data[0].to(device), data[1].to(device)
            labels = labels.type(torch.float)
            #print("Inputs:", inputs.size(), "Labels:", labels.size(), "\n")
            outputs = model(inputs)
            #print("Outputs:", outputs, "Labels:", labels, "\n")

            loss = criterion(outputs.squeeze(), labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            outputs[outputs >= 0.5] = 1
            outputs[outputs < 0.5] = 0
            predicted = outputs

            # Compute accuracy
            acc += (predicted == labels).sum().item()
            count +=len(labels)

            running_loss += loss.item()
            if i % 200 == 199:  # Print every 20 mini-batches
                #for name, param in model.named_parameters():
                #  print(name, param)
                print(predicted.sum())
                accper = acc/count
                print('[%d, %5d] loss: %.3f acc: %.3f' %
                      (epoch + 1, i + 1, running_loss / 200, accper))
                running_loss = 0.0
            # Get predicted classes as the entry with the highest probability

          #score_validation = validation_acc(validation, model(validation))
          #print("Epoch :" epoch, "Score: ", score_validation, "\n")


    print('Finished Training')
    return model

In [ ]:
def test_model(model, loader):
    """
    The testing procedure of the model; it accepts the testing data and the trained model and
    then tests the model on it.

    input: model: torch.nn.Module, the trained model
           loader: torch.data.util.DataLoader, the object containing the testing data

    output: None, the function saves the predictions to a results.txt file
    """
    model.eval()
    predictions = []
    # Iterate over the test data
    with torch.no_grad(): # We don't need to compute gradients for testing
        for [x_batch] in loader:
            x_batch= x_batch.to(device)
            predicted = model(x_batch)
            predicted = predicted.cpu().numpy()
            # Rounding the predictions to 0 or 1
            predicted[predicted >= 0.5] = 1
            predicted[predicted < 0.5] = 0
            #predicted = [i >= 0.5 for i in predicted]
            predictions.append(predicted)
        predictions = np.vstack(predictions)
    np.savetxt("results.txt", predictions, fmt='%i')
    return predictions

In [ ]:
model=train_model(train_loader)

Initial weights of the linear layer:
Parameter containing:
tensor([[-0.0119, -0.0111, -0.0051,  ..., -0.0052, -0.0057,  0.0032],
        [ 0.0020,  0.0070,  0.0078,  ...,  0.0089, -0.0108,  0.0073],
        [ 0.0107,  0.0037, -0.0092,  ...,  0.0040, -0.0121, -0.0093],
        ...,
        [-0.0055,  0.0035, -0.0099,  ..., -0.0122,  0.0113, -0.0016],
        [-0.0085,  0.0115,  0.0122,  ...,  0.0031, -0.0056, -0.0104],
        [-0.0112,  0.0057, -0.0029,  ..., -0.0107, -0.0058,  0.0025]],
       requires_grad=True)
Initial bias of the linear layer:
Parameter containing:
tensor([ 0.0099,  0.0114, -0.0083,  ..., -0.0005,  0.0072,  0.0057],
       requires_grad=True)
Weights after Xavier initialization:
Parameter containing:
tensor([[ 0.0073, -0.0256,  0.0141,  ...,  0.0217, -0.0245, -0.0006],
        [ 0.0079, -0.0226, -0.0252,  ...,  0.0188,  0.0005,  0.0167],
        [-0.0262, -0.0227, -0.0199,  ...,  0.0171, -0.0052, -0.0253],
        ...,
        [ 0.0117, -0.0106,  0.0136,  ..., -0.0

KeyboardInterrupt: 

In [ ]:
test_model(model, test_loader)

array([[1.],
       [1.],
       [1.],
       ...,
       [1.],
       [1.],
       [1.]], dtype=float32)